In [0]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms

In [0]:
n_in=3
n_out=5
n_step=2
b_size=4

In [0]:
rnn = nn.RNNCell(n_in, n_out) # n_input, n_neuronas
X_batch = torch.tensor([[[0,1,2], [3,4,5], #t=0
                         [6,7,8], [9,0,1]], 
                        [[9,8,7], [0,0,0], #t=1
                         [6,5,4], [3,2,1]] 
                       ], dtype = torch.float)
X_batch.shape

torch.Size([2, 4, 3])

In [0]:
hx0 = torch.randn(b_size, n_out) #  N,n_neuronas
hx=hx0
output = []
for i in range(n_step):
    hx = rnn(X_batch[i], hx)
    output.append(hx)
output[0].shape,output[1].shape

(torch.Size([4, 5]), torch.Size([4, 5]))

# MNIST

In [0]:
n_steps = 28
n_in = 28
n_neuronas = 150
n_out = 10
b_size=32
n_capas=1

In [0]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=b_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=b_size, shuffle=False, num_workers=2)

In [0]:
class ImageRNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs,n_capas=1):
        super(ImageRNN, self).__init__()
        #self.rnn = nn.RNN(n_in, n_neuronas,n_capas)
        #self.rnn = nn.LSTM(n_in, n_neuronas,n_capas)
        self.rnn = nn.GRU(n_in, n_neuronas,n_capas)
        self.FC = nn.Linear(n_neuronas, n_out)
    def init_hidden(self, batch):
        # n_capas, b_size, n_neuronas
        return (torch.zeros(n_capas, batch, n_neuronas).cuda())
    def forward(self, X):
        # n_steps, b_size X n_in
        X = X.permute(1, 0, 2) 
        self.hidden = self.init_hidden(X.shape[1])
        #self.state = self.init_hidden(X.shape[1]) #Para LSTM
        out, self.hidden = self.rnn(X, self.hidden)
        #out, (self.hidden, self.state) = self.rnn(X, (self.hidden, self.state)) #Para LSTM
        out = self.FC(self.hidden[-1])
        return out.view(-1, n_out) # b_size, n_output
net = ImageRNN(b_size, n_steps,n_in,n_neuronas,n_out,n_capas)
net.cuda()

ImageRNN(
  (rnn): GRU(28, 150)
  (FC): Linear(in_features=150, out_features=10, bias=True)
)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [0]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0): #Ciclo por Batch
        inputs, labels = data
        inputs=inputs.view(-1,28,28)
        inputs=inputs.cuda()
        labels=labels.cuda()

        optimizer.zero_grad()
        outputs = net(inputs) #Forward
        loss = criterion(outputs, labels) #Loss
        loss.backward() #Backwards
        optimizer.step() #Optim

        running_loss += loss.item()
        if i % 100 == 99:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

[1,   100] loss: 0.087
[1,   200] loss: 0.046
[1,   300] loss: 0.038
[1,   400] loss: 0.035
[1,   500] loss: 0.029
[1,   600] loss: 0.024
[1,   700] loss: 0.020
[1,   800] loss: 0.019
[1,   900] loss: 0.017
[1,  1000] loss: 0.014
[1,  1100] loss: 0.012
[1,  1200] loss: 0.012
[1,  1300] loss: 0.012
[1,  1400] loss: 0.010
[1,  1500] loss: 0.010
[1,  1600] loss: 0.010
[1,  1700] loss: 0.009
[1,  1800] loss: 0.008
[2,   100] loss: 0.007
[2,   200] loss: 0.008
[2,   300] loss: 0.006
[2,   400] loss: 0.007
[2,   500] loss: 0.007
[2,   600] loss: 0.006
[2,   700] loss: 0.006
[2,   800] loss: 0.005
[2,   900] loss: 0.006
[2,  1000] loss: 0.005
[2,  1100] loss: 0.006
[2,  1200] loss: 0.005
[2,  1300] loss: 0.005
[2,  1400] loss: 0.005
[2,  1500] loss: 0.005
[2,  1600] loss: 0.005
[2,  1700] loss: 0.005
[2,  1800] loss: 0.005
[3,   100] loss: 0.004
[3,   200] loss: 0.004
[3,   300] loss: 0.004
[3,   400] loss: 0.005
[3,   500] loss: 0.003
[3,   600] loss: 0.003
[3,   700] loss: 0.003
[3,   800] 

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs=inputs.view(-1,28,28)
        inputs=inputs.cuda()
        labels=labels.cuda()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [0]:
print('Accuracy of the network on the 10000 test images: {} %%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.61 %%
